In [2]:
%matplotlib inline
%matplotlib notebook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ...
The Idea is to define a very tiny dum dataset on witch we can develop eayly our methods

That will help us to define clearly all the necessary methods and to devedie the whole process.


# Dataset

Let's create a dataset of 4 users, and 8 problems
> problems ans users consist only on a list of ids used for as fk in the submission table

In [10]:
users = [ ("U0000%d"%i) for i in range(4)]
pbls = [ ("P0000%d"%i) for i in range(8)]

print users
print pbls

['U00000', 'U00001', 'U00002', 'U00003']
['P00000', 'P00001', 'P00002', 'P00003', 'P00004', 'P00005', 'P00006', 'P00007']


Now we need to generate a dataframe representative of a submissions process.

For now let's focus on those parameters of a submission :
* submission_uid
* user_id
* problem_id
* veredict
* submission_id


In [11]:
headers =  ["submission_uid", "user_id", "problem_id", "veredict", "submission_id"]
verdicts = ["AC", "EE"]

we are goning te generate a submisions list automaticly we random values folowing this rules :

>for each user define a random number of attempts for each problem and a final verdict t that problem (AC or EE)

In [12]:
import random
submission_uid = 0

df = pd.DataFrame(columns=headers)

for u in users:
    for p in pbls:
        attempts = random.randint(0,5)
        submission_id = 0
        for i in range(attempts):
            v = "EE"
            if i == attempts - 1:
                v = random.choice(verdicts)
            sub = ["S%09d"%submission_uid, u, p, v, "S%03d"%submission_id]
            df.loc[len(df)] = sub
            submission_uid+=1
            submission_id+=1

In [109]:
print df

   submission_uid user_id problem_id veredict submission_id
0      S000000000  U00000     P00000       EE          S000
1      S000000001  U00000     P00000       EE          S001
2      S000000002  U00000     P00000       EE          S002
3      S000000003  U00000     P00001       EE          S000
4      S000000004  U00000     P00001       EE          S001
5      S000000005  U00000     P00001       EE          S002
6      S000000006  U00000     P00001       EE          S003
7      S000000007  U00000     P00002       EE          S000
8      S000000008  U00000     P00002       EE          S001
9      S000000009  U00000     P00002       EE          S002
10     S000000010  U00000     P00002       AC          S003
11     S000000011  U00000     P00003       EE          S000
12     S000000012  U00000     P00003       AC          S001
13     S000000013  U00000     P00005       EE          S000
14     S000000014  U00000     P00005       EE          S001
15     S000000015  U00000     P00005    

In [13]:
goodSubs = df[df.veredict == 'AC']
goodSubs.describe()

,submission_uid,user_id,problem_id,veredict,submission_id
count,16,16,16,16,16
unique,16,4,8,1,5
top,S000000057,U00000,P00007,AC,S003
freq,1,5,3,16,6


In [14]:
badSubs = df[df.veredict != 'AC']
badSubs.describe()

,submission_uid,user_id,problem_id,veredict,submission_id
count,68,68,68,68,68
unique,68,4,8,1,5
top,S000000066,U00000,P00003,EE,S000
freq,1,21,14,68,23


In [15]:
table = pd.DataFrame(columns=users , index=pbls)
table

,U00000,U00001,U00002,U00003
P00000,NaN,NaN,NaN,NaN
P00001,NaN,NaN,NaN,NaN
P00002,NaN,NaN,NaN,NaN
P00003,NaN,NaN,NaN,NaN
P00004,NaN,NaN,NaN,NaN
P00005,NaN,NaN,NaN,NaN
P00006,NaN,NaN,NaN,NaN
P00007,NaN,NaN,NaN,NaN


In [91]:
def checkSub(uid, pid):
    p = goodSubs[(goodSubs.problem_id == pid) & 
             (goodSubs.user_id == uid)]
    n = p.count().user_id
    return n if n else float('NaN')

def checkSub2(uid, pid):
    p = df[(df.problem_id == pid) & 
             (df.user_id == uid)]
    n = p.count().user_id
    return n if n else float('NaN')

In [92]:
for u in users:
    table[u] = map(lambda x : checkSub2(u,x), table.index)

In [100]:
table

,U00000,U00001,U00002,U00003
P00000,3.0,1.0,2.0,NaN
P00001,4.0,5.0,3.0,1.0
P00002,4.0,1.0,1.0,5.0
P00003,2.0,5.0,4.0,5.0
P00004,NaN,NaN,4.0,3.0
P00005,5.0,4.0,3.0,3.0
P00006,4.0,NaN,NaN,1.0
P00007,4.0,2.0,4.0,1.0


# First iteration

In [94]:
abilities = pd.Series(index=table.columns)

In [95]:
abilities[table.columns[0]] = table[table.columns[0]].sum()

In [96]:
for c in table.columns:
    abilities[c] = table[c].sum()

In [97]:
abilities

U00000    26.0
U00001    18.0
U00002    21.0
U00003    19.0
dtype: float64

In [98]:
difficulties = pd.Series(index=table.index)
difficulties

P00000   NaN
P00001   NaN
P00002   NaN
P00003   NaN
P00004   NaN
P00005   NaN
P00006   NaN
P00007   NaN
dtype: float64

In [102]:
table.loc[table.index[0]]

U00000    3.0
U00001    1.0
U00002    2.0
U00003    NaN
Name: P00000, dtype: float64

In [103]:
for i in table.index:
    difficulties[i] = table.loc[i].sum()

In [104]:
difficulties

P00000     6.0
P00001    13.0
P00002    11.0
P00003    16.0
P00004     7.0
P00005    15.0
P00006     5.0
P00007    11.0
dtype: float64

In [105]:
plop = pd.DataFrame(columns=difficulties.index)

In [106]:
plop.loc[len(plop)] = difficulties

In [107]:
plop

,P00000,P00001,P00002,P00003,P00004,P00005,P00006,P00007
0,6.0,13.0,11.0,16.0,7.0,15.0,5.0,11.0
